<a href="https://colab.research.google.com/github/dinirst/stroke-fknn-ga/blob/main/SainsKom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# data preparation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pprint
import operator

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics as metrics

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn import preprocessing

In [ ]:
data = pd.read_excel('/content/drive/My Drive/Siscer/stroke.xlsx')
print(data.head())

   gender   age  hypertension  heart_disease  ever_married  work_type  \
0       1  67.0             0              1             1          0   
1       1  80.0             0              1             1          0   
2       0  49.0             0              0             1          0   
3       0  79.0             1              0             1          1   
4       1  81.0             0              0             1          0   

   Residence_type  avg_glucose_level   bmi  smoking_status  stroke  
0               1             228.69  36.6               2       1  
1               0             105.92  32.5               1       1  
2               1             171.23  34.4               3       1  
3               0             174.12  24.0               1       1  
4               1             186.21  29.0               2       1  


**EDA**

In [ ]:
# Display basic information about the DataFrame
print(data.info())

print(data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4981 entries, 0 to 4980
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             4981 non-null   int64  
 1   age                4981 non-null   float64
 2   hypertension       4981 non-null   int64  
 3   heart_disease      4981 non-null   int64  
 4   ever_married       4981 non-null   int64  
 5   work_type          4981 non-null   int64  
 6   Residence_type     4981 non-null   int64  
 7   avg_glucose_level  4981 non-null   float64
 8   bmi                4981 non-null   float64
 9   smoking_status     4981 non-null   int64  
 10  stroke             4981 non-null   int64  
dtypes: float64(3), int64(8)
memory usage: 428.2 KB
None
            gender          age  hypertension  heart_disease  ever_married  \
count  4981.000000  4981.000000   4981.000000    4981.000000   4981.000000   
mean      0.416382    43.419859      0.096165       0.05

In [ ]:
X = data.iloc[:,0:10]
y = data.iloc[:,10]

#split dataset 70:30
xTrain, xTest, yTrain, yTest = train_test_split(X,y,test_size=0.30)
xTrain = xTrain.to_numpy()
xTest = xTest.to_numpy()
yTrain = yTrain.to_numpy()
yTest = yTest.to_numpy()

**FuzzyKNN**

In [ ]:
class FuzzyKNN(BaseEstimator, ClassifierMixin):
    #utk deklrasi kebutuhan k
    def __init__(self, k=3, plot=False):
        self.k = k
        self.plot = plot

#utk train model
    def fit(self, X, y=None):
        self._check_params(X, y)
        self.X = X
        self.y = y

        self.xdim = len(self.X[0])
        self.n = len(y)

        classes = list(set(y))
        classes.sort()
        self.classes = classes

        self.df = pd.DataFrame(self.X)
        self.df['y'] = self.y

        self.memberships = self._compute_memberships()
  #validasi hasil pred
        self.df['membership'] = self.memberships

        self.fitted_ = True
        return self
#untuk memprediksi (pred dgn knn)
    def predict(self, X):
        if not hasattr(self, 'fitted_') or not self.fitted_:
            raise Exception('predict() called before fit()')
        else:
            m = 2
            y_pred = []
            epsilon = 1e-9  # Small constant to prevent division by zero

            for x in X:
              #model dipanggil
                neighbors = self._find_k_nearest_neighbors(pd.DataFrame.copy(self.df), x)

                votes = {}
                for c in self.classes:
                    den = 0
                    for n in range(self.k):
                        dist = np.linalg.norm(x - neighbors.iloc[n, 0:self.xdim]) + epsilon
                        den += 1 / (dist ** (2 / (m-1)))

                    neighbors_votes = []
                    for n in range(self.k):
                        dist = np.linalg.norm(x - neighbors.iloc[n, 0:self.xdim]) + epsilon
                        num = (neighbors.iloc[n].membership[c]) / (dist ** (2 / (m-1)))

                        vote = num / den
                        neighbors_votes.append(vote)
                    votes[c] = np.sum(neighbors_votes)

                pred = max(votes.items(), key=operator.itemgetter(1))[0]
                y_pred.append((pred, votes))

            return y_pred
#utk menghitung score accuracy
    def score(self, X, y):
        if not hasattr(self, 'fitted_') or not self.fitted_:
            raise Exception('score() called before fit()')
        else:
            predictions = self.predict(X)
            y_pred = [t[0] for t in predictions]

            return accuracy_score(y_pred=y_pred, y_true=y)
#model knn (perhitungan knn manual)
    def _find_k_nearest_neighbors(self, df, x):
        X = df.iloc[:, 0:self.xdim].values

        df['distances'] = [np.linalg.norm(X[i] - x) for i in range(self.n)]

        df.sort_values(by='distances', ascending=True, inplace=True)
        neighbors = df.iloc[0:self.k]

        return neighbors

#menghitung tetangga terdekat, yg nilanya 1 brp 0 brp
    def _get_counts(self, neighbors):
        groups = neighbors.groupby('y')
        counts = {group[1]['y'].iloc[0]: group[1].count()[0] for group in groups}

        return counts
#utk menghitung derajat keanggotaan hasil prediksi knn
    def _compute_memberships(self):
        memberships = []
        for i in range(self.n):
            x = self.X[i]
            y = self.y[i]

            neighbors = self._find_k_nearest_neighbors(pd.DataFrame.copy(self.df), x)
            counts = self._get_counts(neighbors)

            membership = dict()
            for c in self.classes:
                try:
                    uci = 0.49 * (counts[c] / self.k)
                    if c == y:
                        uci += 0.51
                    membership[c] = uci
                except:
                    membership[c] = 0

            memberships.append(membership)
        return memberships
#cek k, apakah nilai k sudah tepat
    def _check_params(self, X, y):
        if type(self.k) != int:
            raise Exception('"k" should have type int')
        elif self.k >= len(y):
            raise Exception('"k" should be less than no of feature sets')
        elif self.k % 2 == 0:
            raise Exception('"k" should be odd')

        if type(self.plot) != bool:
            raise Exception('"plot" should have type bool')

In [ ]:
#Fuzzy KNN
fkModel = FuzzyKNN()
#training
fkModel.fit(xTrain, yTrain)
#memprediksi apakah test yg 30% hasilnya apa, apakah sdh sesuai
y_predFK=fkModel.predict(xTest)

#Hilangkan indeks pertama
new_y_predFK = []
for item in y_predFK :
    new_y_predFK.append(item[1])
y_predFK = new_y_predFK

#ubah ke ndarray (utk rubah tipe data disamakan)
new_y_predFK = []
for item in y_predFK :
    new_y_predFK.append([item[0],item[1]])
fk_proba = np.array(new_y_predFK)

In [ ]:
#akurasi Fuzzy KNN
y_pred_fk = np.argmax(fk_proba, axis=1)
accuracy_score(y_pred_fk,yTest)

0.9391304347826087